In [1]:
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import numpy as np

## Spark Session & H2O Context

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:2.7.3" pyspark-shell'
conf = SparkConf().setAppName("Medhere").setMaster("local[*]")

# Extra memory.
SparkContext.setSystemProperty('spark.executor.memory', '15g')
SparkContext.setSystemProperty('spark.driver.memory', '15g')

# Instantiate Spark session.
sc = pyspark.SparkContext(conf=conf).getOrCreate()
ss = SparkSession.builder.getOrCreate()

In [3]:
ss

In [4]:
from pysparkling import *
import h2o
from h2o.automl import H2OAutoML

# Instantiate h2o context.
hc = H2OContext.getOrCreate()

Connecting to H2O server at http://10.235.122.168:54323 ... successful.


H2O_cluster_uptime:,10 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,sparkling-water-lisachua_local-1592731683245
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,881 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.30.0.4-1-2.4
 * H2O name: sparkling-water-lisachua_local-1592731683245
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,10.235.122.168,54323)
  ------------------------

  Open H2O Flow in browser: http://10.235.122.168:54325 (CMD + click in Mac OSX)

    


## Upload data to h2o frame

In [5]:
df = h2o.upload_file("h2o_data/train.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
test_df = h2o.upload_file("h2o_data/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
target_col = "intoxicated"

# Change target column to enum type.
df[target_col] = df[target_col].asfactor()
df.describe()

Rows:61642
Cols:136




,x_mean,x_median,x_min,x_max,x_std,x_percentile_5,x_percentile_25,x_percentile_75,x_percentile_95,x_zero_crossing,y_mean,y_median,y_min,y_max,y_std,y_percentile_5,y_percentile_25,y_percentile_75,y_percentile_95,y_zero_crossing,z_mean,z_median,z_min,z_max,z_std,z_percentile_5,z_percentile_25,z_percentile_75,z_percentile_95,z_zero_crossing,x_mean_mean,x_mean_median,x_mean_min,x_mean_max,x_mean_std,x_median_mean,x_median_median,x_median_min,x_median_max,x_median_std,x_min_mean,x_min_median,x_min_min,x_min_max,x_min_std,x_max_mean,x_max_median,x_max_min,x_max_max,x_max_std,x_std_mean,x_std_median,x_std_min,x_std_max,x_std_std,y_mean_mean,y_mean_median,y_mean_min,y_mean_max,y_mean_std,y_median_mean,y_median_median,y_median_min,y_median_max,y_median_std,y_min_mean,y_min_median,y_min_min,y_min_max,y_min_std,y_max_mean,y_max_median,y_max_min,y_max_max,y_max_std,y_std_mean,y_std_median,y_std_min,y_std_max,y_std_std,z_mean_mean,z_mean_median,z_mean_min,z_mean_max,z_mean_std,z_median_mean,z_median_median,z_median_min,z_median_max,z_median_std,z_min_mean,z_min_median,z_min_min,z_min_max,z_min_std,z_max_mean,z_max_median,z_max_min,z_max_max,z_max_std,z_std_mean,z_std_median,z_std_min,z_std_max,z_std_std,x_mean_last3,x_median_last3,x_min_last3,x_max_last3,x_std_last3,y_mean_last3,y_median_last3,y_min_last3,y_max_last3,y_std_last3,z_mean_last3,z_median_last3,z_min_last3,z_max_last3,z_std_last3,x_mean_first3,x_median_first3,x_min_first3,x_max_first3,x_std_first3,y_mean_first3,y_median_first3,y_min_first3,y_max_first3,y_std_first3,z_mean_first3,z_median_first3,z_min_first3,z_max_first3,z_std_first3,intoxicated
type,real,real,real,real,real,real,real,real,real,int,real,real,real,real,real,real,real,real,real,int,real,real,real,real,real,real,real,real,real,int,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,enum
mins,-10.512938999997443,-12.429498,-43.33507,-10.512939,0.0,-16.073576,-12.703634,-10.512939,-10.512939,0.0,-180900784.0016,-180900784.0,-180900800.0,-180900784.0,0.0,-180900784.0,-180900784.0,-180900784.0,-180900784.0,0.0,-38.90190499999542,-38.901905,-45.46259000000001,-38.901905,0.0,-38.901905,-38.901905,-38.901905,-38.901905,0.0,-10.512939000000143,-10.512939000000143,-12.703633999999951,-10.512939000000143,0.0,-10.512939,-11.040861,-12.703634,-10.512939,0.0,-20.5706808,-20.734434,-43.33507,-14.522783,0.0,-10.512939,-10.512939,-12.703634,-10.512939,0.0,0.0,0.0,0.0,0.0,0.0,-180900784.0016,-180900784.0,-180900784.016,-180900784.0,0.0,-180900784.0,-180900784.0,-180900784.0,-180900784.0,0.0,-180900800.0,-180900800.0,-180900800.0,-180900800.0,0.0,-180900784.0,-180900784.0,-180900784.0,-180900784.0,0.0,0.0,0.0,0.0,0.0,0.0,-38.901904999999424,-38.90190499999942,-38.90190499999942,-38.90190499999942,0.0,-38.901905,-38.901905,-38.901905,-38.901905,0.0,-38.901905,-38.901905,-45.46259000000001,-38.901905,0.0,-38.901905,-38.901905,-38.901905,-38.901905,0.0,0.0,0.0,0.0,0.0,0.0,-12.005472445499995,-12.703634,-22.61478633333333,-10.512939,0.0,-180900784.00533333,-180900784.0,-180900800.0,-180900784.0,0.0,-38.90190499999942,-38.901905,-39.581234,-38.901905,0.0,-12.571953000000102,-12.571953,-24.556833,-12.571953,0.0,-180900784.0,-180900784.0,-180900800.0,-180900784.0,0.0,-38.90190499999942,-38.901905,-40.32352233333333,-38.901905,0.0,
mean,-0.027575762414379392,-0.02874816690475351,-0.4484978418570657,0.4142208711316444,0.1092774522613886,-0.18694669431440533,-0.08142778677115299,0.025699725929142072,0.13425419938663463,51.53012556373899,-5658193.859985733,-5658101.42264166,-5684

In [8]:
df[target_col].unique()

C1
0
1


In [9]:
# Get list of features.
features = df.names[:]
features.remove(target_col)

## AutoML Leaderboard

The models are ranked by a default metric based on the problem type. In binary classification problems, that metric is AUC.

In [10]:
# H2O AutoML hyperparameters
max_models = 10
seed = 1 # Set seed for reproducibility.
nfolds = 10 # 10-fold cross-validation for model performance.

In [11]:
model_automl = H2OAutoML(max_models=max_models, seed=seed, nfolds=nfolds)
model_automl.train(x=features,
                   y=target_col,
                   training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [12]:
# Leaderboard
model_automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20200621_022838,0.94835,0.285581,0.912156,0.13287,0.301165,0.0907006
StackedEnsemble_BestOfFamily_AutoML_20200621_022838,0.94689,0.288522,0.909615,0.134066,0.302852,0.0917195
XGBoost_3_AutoML_20200621_022838,0.943403,0.280377,0.904684,0.13831,0.304328,0.0926154
GBM_4_AutoML_20200621_022838,0.943138,0.288151,0.904152,0.138551,0.304627,0.0927976
XGBoost_1_AutoML_20200621_022838,0.94252,0.283892,0.902815,0.140359,0.306527,0.0939587
DRF_1_AutoML_20200621_022838,0.942081,0.310042,0.90101,0.140959,0.311623,0.0971091
XGBoost_2_AutoML_20200621_022838,0.941368,0.287794,0.90097,0.1418,0.308407,0.0951152
GBM_3_AutoML_20200621_022838,0.940601,0.296633,0.899606,0.141005,0.30809,0.0949194
GBM_5_AutoML_20200621_022838,0.938592,0.301756,0.896721,0.146552,0.311411,0.0969769
GBM_2_AutoML_20200621_022838,0.937874,0.304539,0.895329,0.146394,0.311861,0.0972574


## Lead Model (Stacked Ensemble)

In [33]:
# Predict with leading model, get model performance.
model_automl.leader.model_performance(test_df)


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.0951810997830157
RMSE: 0.30851434291296037
LogLoss: 0.2964981746841264
Null degrees of freedom: 10878
Residual degrees of freedom: 10872
Null deviance: 14172.327317342631
Residual deviance: 6451.207284777223
AIC: 6465.207284777223
AUC: 0.944610487752468
AUCPR: 0.9065473572738116
Gini: 0.8892209755049361

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.34142004598793446: 


,,0,1,Error,Rate
0,0,6016.0,985.0,0.1407,(985.0/7001.0)
1,1,540.0,3338.0,0.1392,(540.0/3878.0)
2,Total,6556.0,4323.0,0.1402,(1525.0/10879.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.341420,0.814047,244.0
1,max f2,0.097198,0.887903,338.0
2,max f0point5,0.753553,0.827740,94.0
3,max accuracy,0.452964,0.862763,207.0
4,max precision,0.978258,1.000000,0.0
5,max recall,0.030143,1.000000,383.0
6,max specificity,0.978258,1.000000,0.0
7,max absolute_mcc,0.448758,0.705214,208.0
8,max min_per_class_accuracy,0.345000,0.859591,243.0
9,max mean_per_class_accuracy,0.294961,0.860917,259.0



Gains/Lift Table: Avg response rate: 35.65 %, avg score: 35.79 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010019,0.978007,2.805312,2.805312,1.000000,0.978307,1.000000,0.978307,0.028107,0.028107,180.531202,180.531202
1,,2,0.022520,0.977922,2.805312,2.805312,1.000000,0.977922,1.000000,0.978093,0.035070,0.063177,180.531202,180.531202
2,,3,0.047982,0.975439,2.805312,2.805312,1.000000,0.975640,1.000000,0.976792,0.071429,0.134605,180.531202,180.531202
3,,4,0.050005,0.975043,2.805312,2.805312,1.000000,0.975257,1.000000,0.976729,0.005673,0.140278,180.531202,180.531202
4,,5,0.100009,0.967125,2.800155,2.802734,0.998162,0.970830,0.999081,0.973780,0.140021,0.280299,180.015519,180.273360
5,,6,0.150014,0.924986,2.676391,2.760620,0.954044,0.955335,0.984069,0.967632,0.133832,0.414131,167.639143,176.061955
6,,7,0.224101,0.858218,2.150971,2.559071,0.766749,0.870358,0.912223,0.935473,0.159360,0.573491,115.097125,155.907052
7,,8,0.301866,0.653065,1.943159,2.400404,0.692671,0.743089,0.855664,0.885912,0.151109,0.724600,94.315939,140.040401
8,,9,0.400037,0.330663,1.415789,2.158775,0.504682,0.515962,0.769531,0.795125,0.138989,0.863589,41.578949,115.877526
9,,10,0.500046,0.144999,0.938542,1.914729,0.334559,0.212425,0.682537,0.678585,0.093863,0.957452,-6.145811,91.472859


In [27]:
# Calculate accuracy and F1 score.
print(f"Accuracy: {(3338 + 6016)/(6556+4323)}") # Accu = (TP + TN) / Total
print(f"F1 Score: {(2*3338)/((2*3338)+985+540)}") # F1 = 2TP / (2TP + FP + FN)

Accuracy: 0.8598216747862855
F1 Score: 0.8140470674308011


In [18]:
# Get base models for leading stacked ensemble.
model_automl.leader.params.keys()
model_automl.leader.params['base_models']

{'default': [],
 'actual': [{'__meta': {'schema_version': 3,
    'schema_name': 'KeyV3',
    'schema_type': 'Key<Keyed>'},
   'name': 'XGBoost_3_AutoML_20200621_022838',
   'type': 'Key<Keyed>',
   'URL': None},
  {'__meta': {'schema_version': 3,
    'schema_name': 'KeyV3',
    'schema_type': 'Key<Keyed>'},
   'name': 'GBM_4_AutoML_20200621_022838',
   'type': 'Key<Keyed>',
   'URL': None},
  {'__meta': {'schema_version': 3,
    'schema_name': 'KeyV3',
    'schema_type': 'Key<Keyed>'},
   'name': 'XGBoost_1_AutoML_20200621_022838',
   'type': 'Key<Keyed>',
   'URL': None},
  {'__meta': {'schema_version': 3,
    'schema_name': 'KeyV3',
    'schema_type': 'Key<Keyed>'},
   'name': 'DRF_1_AutoML_20200621_022838',
   'type': 'Key<Keyed>',
   'URL': None},
  {'__meta': {'schema_version': 3,
    'schema_name': 'KeyV3',
    'schema_type': 'Key<Keyed>'},
   'name': 'XGBoost_2_AutoML_20200621_022838',
   'type': 'Key<Keyed>',
   'URL': None},
  {'__meta': {'schema_version': 3,
    'schema_name'

## Lead XGB

In [29]:
m = h2o.get_model("XGBoost_3_AutoML_20200621_022838")

In [31]:
# Model performance.
m.model_performance(test_df)


ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.09663452581528809
RMSE: 0.3108609428913322
LogLoss: 0.28996912464882935
Mean Per-Class Error: 0.14628260944671645
AUC: 0.9386788957210048
AUCPR: 0.8975077052431302
Gini: 0.8773577914420096

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.40983649094899494: 


,,0,1,Error,Rate
0,0,6008.0,993.0,0.1418,(993.0/7001.0)
1,1,586.0,3292.0,0.1511,(586.0/3878.0)
2,Total,6594.0,4285.0,0.1451,(1579.0/10879.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.409836,0.806566,211.0
1,max f2,0.188212,0.881124,292.0
2,max f0point5,0.704798,0.817838,96.0
3,max accuracy,0.464692,0.855777,191.0
4,max precision,0.997682,1.000000,0.0
5,max recall,0.013457,1.000000,379.0
6,max specificity,0.997682,1.000000,0.0
7,max absolute_mcc,0.409836,0.693086,211.0
8,max min_per_class_accuracy,0.395266,0.853017,216.0
9,max mean_per_class_accuracy,0.399941,0.853717,214.0



Gains/Lift Table: Avg response rate: 35.65 %, avg score: 35.55 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010019,0.997652,2.805312,2.805312,1.000000,0.997878,1.000000,0.997878,0.028107,0.028107,180.531202,180.531202
1,,2,0.031069,0.997589,2.805312,2.805312,1.000000,0.997589,1.000000,0.997682,0.059051,0.087158,180.531202,180.531202
2,,3,0.047615,0.996024,2.805312,2.805312,1.000000,0.996054,1.000000,0.997116,0.046416,0.133574,180.531202,180.531202
3,,4,0.056899,0.995872,2.805312,2.805312,1.000000,0.995872,1.000000,0.996913,0.026044,0.159618,180.531202,180.531202
4,,5,0.100469,0.985254,2.793475,2.800179,0.995781,0.992541,0.998170,0.995017,0.121712,0.281331,179.347526,180.017878
5,,6,0.150014,0.895115,2.617944,2.739992,0.933210,0.939389,0.976716,0.976645,0.129706,0.411037,161.794424,173.999225
6,,7,0.200018,0.769853,2.186493,2.601617,0.779412,0.801721,0.927390,0.932914,0.109335,0.520371,118.649319,160.161749
7,,8,0.300028,0.592174,1.848721,2.350652,0.659007,0.696249,0.837929,0.854026,0.184889,0.705260,84.872125,135.065207
8,,9,0.400037,0.389486,1.487744,2.134925,0.530331,0.514482,0.761029,0.769140,0.148788,0.854048,48.774360,113.492495
9,,10,0.500046,0.218137,0.961748,1.900289,0.342831,0.308938,0.677390,0.677099,0.096184,0.950232,-3.825241,90.028948


In [32]:
# Calculate accuracy and F1 score.
print(f"Accuracy: {(3292 + 6008)/(6594+4285)}") # Accu = (TP + TN) / Total
print(f"F1 Score: {(2*3292)/((2*3292)+993+586)}") # F1 = 2TP / (2TP + FP + FN)

Accuracy: 0.8548579832705212
F1 Score: 0.8065662134019356


In [16]:
params_list = ['ntrees', 'learn_rate', 'max_depth', 'booster', 
               'min_child_weight', 'subsample', 'colsample_bytree',
               'reg_lambda']
for p in params_list:
    print(f"{p}: {m.params[p].get('actual')}")

ntrees: 136
learn_rate: 0.3
max_depth: 5
booster: gbtree
min_child_weight: 3.0
subsample: 0.8
colsample_bytree: 0.8
reg_lambda: 1.0
